In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
%matplotlib inline

---
## 外れ値の削除
---

In [ ]:
A=pd.read_csv('./train.csv')
numerical_var=A.dtypes[A.dtypes !='object'].index
numerical_var

In [ ]:
var='LotArea'
A.plot.scatter(x=var,y='SalePrice',ylim=(0,800000))

In [ ]:
A.drop(A[A['LotArea']>=100000].index,inplace=True)
A.plot.scatter(x=var,y='SalePrice',ylim=(0,800000))

---
## 欠損値処理
---

In [ ]:
A=pd.read_csv('./train.csv')
for i in A.columns:
    if A[i].isnull().sum()>0:
        print('{0}: {1}'.format(i,A[i].isnull().sum()))

#### 処理方法： *LotFrontage* を例として

In [ ]:
A=pd.read_csv('./train.csv')
#中位数
A.loc[A['LotFrontage'].isnull()==True,'LotFrontage']=A['LotFrontage'].median()

In [ ]:
A=pd.read_csv('./train.csv')
#平均数
A.loc[A['LotFrontage'].isnull()==True,'LotFrontage']=A['LotFrontage'].mean()

In [ ]:
A=pd.read_csv('./train.csv')
#最頻値
A.loc[A['LotFrontage'].isnull()==True,'LotFrontage']=A['LotFrontage'].mode()

In [ ]:
A=pd.read_csv('./train.csv')
#0
A.loc[A['LotFrontage'].isnull()==True,'LotFrontage']=0

In [ ]:
A=pd.read_csv('./train.csv')
#カラム削除
A.drop('LotFrontage',axis=1,inplace=True)
#などなど

---
## 追加変数
---

### 量的変数のbin
#### 地域別物件価格中位数を用いて分類する

In [ ]:
A=pd.read_csv('./train.csv')
neighbor_price=dict(A['SalePrice'].groupby(A['Neighborhood']).median())
A['neighbor_bins']=0

In [ ]:
for i in neighbor_price:
    if neighbor_price[i]<=100000:
        neighbor_price[i]=1
    elif (neighbor_price[i]>100000) & (neighbor_price[i]<=200000):
        neighbor_price[i]=2
    elif (neighbor_price[i]>200000) & (neighbor_price[i]<=300000):
        neighbor_price[i]=3
    else:
        neighbor_price[i]=4
        
for i in neighbor_price:
    A.loc[A['Neighborhood']==i,'neighbor_bins']=neighbor_price[i]

In [ ]:
A.loc[:,['Neighborhood','neighbor_bins']].head(5)

---
### 質的変数のカテゴリーの統合
---

In [ ]:
A=pd.read_csv('./train.csv')
obj_var=A.dtypes[A.dtypes =='object'].index
obj_var

In [ ]:
A['LandSlope'].value_counts()

In [ ]:
A.loc[A['LandSlope']!='Gtl','LandSlope']='NoGtl'
A['LandSlope'].value_counts()

---
## 正規化
---

In [ ]:
A=pd.read_csv('./train.csv')
B=pd.DataFrame()
B['SalePrice']=A['SalePrice']
B['SalePrice']=np.log1p(B['SalePrice'])    #対数をとる：　log(1+x); 指数関数 exp(x)-1 のコマンド：　np.expm1 ; log,expを使っても構わない
D=B

scaler = StandardScaler()
scaler.fit(D)
C=pd.DataFrame(scaler.transform(D))
C.columns=['SalePrice']

fig,ax=plt.subplots(1,3,figsize=(15,5))
sns.distplot(A['SalePrice'],fit=norm,ax=ax[0])
sns.distplot(B['SalePrice'],fit=norm,ax=ax[1])
sns.distplot(C['SalePrice'],fit=norm,ax=ax[2])